#### Acknowledgement to the faculty who taught the Optimisation subject at Jio Institute for helping with the formulation discussions and guidance

# Delivery Assignment Problem

In [1]:
from ortools.linear_solver import pywraplp
import pandas as pd

In [2]:
orders_df = pd.read_csv('/Users/maulikruparel/Documents/AIandDS/RandomPractice/optimisation/data/Sample_Orders.csv')

In [3]:
orders_df.head()

,store_id,store_latitude,store_longitude,customer_latitude,customer_longitude,order_time,order_id,time_promise,ord_packing_time,price,distance
0,1,12.988047,77.598497,13.016825,77.578845,18-Mar-2024 12:22PM,88184699,34,11,325,3.843544
1,1,12.988047,77.598497,12.992958,77.607472,18-Mar-2024 12:19PM,74788581,26,12,578,1.115243
2,1,12.988047,77.598497,13.006920,77.612363,18-Mar-2024 12:19PM,50443196,29,10,279,2.580831
3,1,12.988047,77.598497,13.015495,77.592164,18-Mar-2024 12:22PM,74272493,29,12,645,3.128245
4,1,12.988047,77.598497,12.973296,77.605567,18-Mar-2024 12:20PM,99415675,16,9,433,1.810360


In [4]:
executives_df = pd.read_csv('/Users/maulikruparel/Documents/AIandDS/RandomPractice/optimisation/data/Sample_Executives.csv')

In [5]:
executives_df.sample(5)

,de_id,de_latitude,de_longitude,de_time_to_free
163,67584926,12.955229,77.562548,0
96,67584993,12.937388,77.624469,4
138,67584951,12.964309,77.606223,9
118,67584971,12.973788,77.574160,5
142,67584947,12.941577,77.570455,1


## OR-tools based

In [6]:
# Create the solver instance
solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    raise Exception("Solver not found!")


## Decision Variable
**\( X_{ij} \):** Binary decision variable where:
- \( X_{ij} = 1 \): Order \( i \) is assigned to delivery executive \( j \).
- \( X_{ij} = 0 \): Order \( i \) is not assigned to delivery executive \( j \).


In [60]:
num_executives = len(executives_df)
num_orders = len(orders_df)

In [9]:
x = {}
for i in range(num_orders):
    for j in range(num_executives):
        x[i, j] = solver.BoolVar(f'x[{i},{j}]')

## Objective Function:
The objective is to **minimize the total cost**, which consists of three components:

### 1. Cost of Delivery:
$$
\text{Cost of Delivery} = f(d_{ij}) + f(t_{ij})
$$
- $ f(d_{ij}) $: Cost based on the distance between order $ i $ and delivery executive $ j $.
- $ f(t_{ij}) $: Cost based on the time taken to deliver order $ i $ by delivery executive $ j $ (e.g., wait time).

---

### 2. Customer Experience:
**Customer Experience** = f(delay<sub>ij</sub>)

where:

* **delay<sub>ij</sub> = delivery time - promise time**: The delay for order *i* assigned to delivery executive *j*.
* **f(delay<sub>ij</sub>)**: A penalty function that increases with the delay, reflecting the impact on customer satisfaction.

### 3. Unassignment Penalty:

**Unassignment Penalty** = [1 - ∑<sub>j</sub> X<sub>ij</sub>] * f(all delays<sub>ij</sub>)

where:

* **∑<sub>j</sub> X<sub>ij</sub>**: Sum of assignments for order *i* across all delivery executives *j*. If ∑<sub>j</sub> X<sub>ij</sub> = 0, the order is unassigned.
* **[1 - ∑<sub>j</sub> X<sub>ij</sub>]**: Equals 1 if the order is unassigned, 0 otherwise.
* **f(all delays<sub>ij</sub>)**: A penalty function applied to unassigned orders, reflecting the cost of leaving an order unfulfilled.
---

### Final Objective Function:
**Minimize:** 

∑<sub>i,j</sub> X<sub>ij</sub> * (f(d<sub>ij</sub>) + f(t<sub>ij</sub>) + f(delay<sub>ij</sub>)) + ∑<sub>i</sub> [1 - ∑<sub>j</sub> X<sub>ij</sub>] * f(all delays<sub>ij</sub>)

In [10]:
# Assumptions
cost_per_km = 2             # Cost per kilometer
speed_km_per_min = 0.5      # Speed of travel in km/min
wait_time_cost = 1          # Cost per min

# Helper function to calculate haversine distance
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points on the Earth.
    """
    R = 6371  # Radius of Earth in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c  # Distance in km

In [11]:
# Cost linked to the distance required to execute the order delivery 
distance_cost = np.zeros((num_orders,num_executives))

# Cost linked to the waiting period asosciated with an order & delivery executive in case delivery exec. arrives at store earlier than the order pacaking time.
time_cost = np.zeros((num_orders,num_executives)) #waiting time related cost

# Time needed to complete an order by a given delivery executive.
order_time_matrix = np.zeros((num_orders,num_executives))

#Penalty associated with the order and a given delivery executive in case the order delivery time exceed the time promised.
delay_penalty = np.zeros((num_orders,num_executives))

# Cost associated with the order in case a delivery executive could not be assigned.
unassignment_cost = np.zeros((num_orders,1))

In [12]:
for j, exec_row in executives_df.iterrows():
    for i, order_row in orders_df.iterrows():
        
        # Distance from executive to store
        dist_exec_to_store = haversine(
                                        exec_row["de_latitude"], exec_row["de_longitude"],
                                        order_row["store_latitude"], order_row["store_longitude"]
                                    )
        
        # Distance from store to customer
        dist_store_to_customer = haversine(
                                            order_row["store_latitude"], order_row["store_longitude"],
                                            order_row["customer_latitude"], order_row["customer_longitude"]
                                        )
        
        total_distance = dist_exec_to_store + dist_store_to_customer
        
        '''########## Filling the matrices values #####################'''


        # Distance cost matrix
        distance_cost[i, j] = cost_per_km * total_distance        

        
        #Time cost matrix
        time_exec_to_store = exec_row['de_time_to_free'] + (dist_exec_to_store / speed_km_per_min)
        time_store_to_customer = dist_store_to_customer / speed_km_per_min
    
        ord_packing_time = order_row['ord_packing_time']
        time_cost[i,j] = max(0, ord_packing_time - time_exec_to_store) * wait_time_cost

        
        # Order delivery time matrix
        order_time_matrix[i,j] = time_exec_to_store + max(0, ord_packing_time - time_exec_to_store) + time_store_to_customer
        
        # Delay incurred = order_delivery_time - time promised
        time_promise = order_row['time_promise']
        delay = time_exec_to_store + max(0, ord_packing_time - time_exec_to_store) + time_store_to_customer - time_promise

        delay_penalty[i,j] = 100/delay # more delayed a customer, lesser the cost so that largest delayed order gets assigned first at a particular time instant of decision making.

# unassignment penalty matrix
unassignment_cost = orders_df.price.to_numpy().reshape((num_orders,1))
        

In [13]:
# Creating objective function instance
objective = solver.Objective()

In [14]:
# Building the objective function for minimizing total cost
objective = solver.Sum(
    x[i, j] * (distance_cost[i, j] + time_cost[i, j] + delay_penalty[i, j])
    for i in range(num_orders)
    for j in range(num_executives)
)

# Add unassignment penalty to the objective
unassignment_penalty = solver.Sum(
    (1 - solver.Sum(x[i, j] for j in range(num_executives))) * unassignment_cost[i].item()
    for i in range(num_orders)
)

# Combine the two components into the final objective
solver.Minimize(objective + unassignment_penalty)

## Constraints:

1. **One Order Per Delivery Executive:**

   ∑<sub>i</sub> X<sub>ij</sub> = 1 ∀ j

   - Each delivery executive *j* must be assigned exactly one order.

2. **One Delivery Executive Per Order:**

   ∑<sub>j</sub> X<sub>ij</sub> ≤ 1 ∀ i

   - Each order *i* can be assigned to at most one delivery executive.

In [15]:
# Constraints
# 1. Each order is assigned to only one executive
for i in range(num_orders):
    solver.Add(solver.Sum(x[i, j] for j in range(num_executives)) <= 1)

# 2. Each order is assigned to atmost one executive
for j in range(num_executives):
    solver.Add(solver.Sum(x[i, j] for i in range(num_orders)) == 1)


### Solving

In [20]:
# Solve the problem
solver.Minimize(objective)

status = solver.Solve()

In [54]:
# Prepare the output as a DataFrame
if status == pywraplp.Solver.OPTIMAL:
    print("Optimal solution found!")
    assignments = []
    for i in range(num_orders):
        for j in range(num_executives):
            if x[i, j].solution_value() > 0.5:  # If assigned
                assignments.append({
                    "assigned_executive_id": executives_df.loc[j, "de_id"],
                    "order_id": orders_df.loc[i, "order_id"],
                    "cost_to_company": (distance_cost[i, j] + time_cost[i, j] + delay_penalty[i, j]),
                    "delivery_time": order_time_matrix[i, j] #+ orders_df.loc[i, "ord_packing_time"]
                })

    assignments_df = pd.DataFrame(assignments)
    order_cols = ['store_id','order_time', 'order_id', 'time_promise','ord_packing_time', 'price']
    allocations_df = orders_df[order_cols].merge(assignments_df[['assigned_executive_id','order_id','delivery_time','cost_to_company']],on='order_id',how='left')
    
    allocations_df =allocations_df.convert_dtypes()

    
else:
    assignments_df = pd.DataFrame()
    print("No optimal solution found.")



Optimal solution found!


In [58]:
# review the total cost incurred for all the orders-> value of objective function
solver.Objective().Value()

-2055750.6454373517

In [55]:
#Output results
allocations_df

,store_id,order_time,order_id,time_promise,ord_packing_time,price,assigned_executive_id,delivery_time,cost_to_company
0,1,18-Mar-2024 12:22PM,88184699,34,11,325,<NA>,<NA>,<NA>
1,1,18-Mar-2024 12:19PM,74788581,26,12,578,67585081,14.230482,5.733957
2,1,18-Mar-2024 12:19PM,50443196,29,10,279,67585009,15.161653,7.935357
3,1,18-Mar-2024 12:22PM,74272493,29,12,645,67584991,18.256479,8.948544
4,1,18-Mar-2024 12:20PM,99415675,16,9,433,67585003,15.843999,-632.176594
...,...,...,...,...,...,...,...,...,...
441,150,18-Mar-2024 12:18PM,58311614,38,13,219,<NA>,<NA>,<NA>
442,150,18-Mar-2024 12:20PM,10574214,35,9,525,<NA>,<NA>,<NA>
443,150,18-Mar-2024 12:18PM,56727371,36,8,496,<NA>,<NA>,<NA>
444,150,18-Mar-2024 12:20PM,61060721,31,9,396,67584890,16.741374,3.728075


In [49]:
#number of order which were assigned for delivery
allocations_df.assigned_executive_id.count()

# 300 order were assigned

300

In [39]:
# Review the orders delivery in more than promised time
allocations_df.loc[(allocations_df.time_promise < allocations_df.delivery_time)]

#0 orders-> all orders met time promised scenario

,store_id,store_latitude,store_longitude,customer_latitude,customer_longitude,order_time,order_id,time_promise,ord_packing_time,price,distance,executive_id,delivery_time,cost_to_company


In [34]:
#Order which will remain unassigned due to less number of delivery executive


len(allocations_df.loc[allocations_df.executive_id.isna()])

#146 order out of 446.

146